Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.

In [0]:
df = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & 
        (df['SALE_PRICE'] > 100000) & 
        (df['SALE_PRICE'] < 2000000)]

Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [0]:
split = '04/01/2019'
train = df[(df['SALE_DATE'] < split)]
test = df[(df['SALE_DATE'] >= split)]

One-hot encoding of categorical features.


In [0]:
import category_encoders as ce

In [113]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
BUILDING_CLASS_CATEGORY,2507,1,01 ONE FAMILY DWELLINGS,2507
APARTMENT_NUMBER,1,1,RP.,1
TAX_CLASS_AT_PRESENT,2507,2,1,2476
BOROUGH,2507,5,4,1204
NEIGHBORHOOD,2507,6,OTHER,2382
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919
BUILDING_CLASS_AT_PRESENT,2507,13,A1,919
SALE_DATE,2507,68,01/31/2019,78
LAND_SQUARE_FEET,2507,887,"4,000",234
ADDRESS,2507,2497,130-52 LEFFERTS BOULEVARD,2


In [114]:
# 'BOROUGH' SEEMS LIKE A GOOD CANDIDATE FOR ONE-HOT ENCODING
train.groupby('BOROUGH')['SALE_PRICE'].describe()

,count,mean,std,min,25%,50%,75%,max
BOROUGH,,,,,,,,
1,1.0,739000.000000,NaN,739000.0,739000.0,739000.0,739000.0,739000.0
2,242.0,473104.623967,184647.130460,108000.0,355000.0,465000.0,550000.0,1750000.0
3,398.0,768734.633166,366138.391194,115000.0,523125.0,676421.5,930000.0,1955000.0
4,1204.0,649005.172757,289512.945057,104000.0,450000.0,601500.0,815000.0,1876000.0
5,662.0,537305.802115,217441.137801,113348.0,415000.0,520000.0,617250.0,1850000.0


In [0]:
target = 'SALE_PRICE'

# Q: WHY DON'T WE JUST USE A COUPLE FEATURES LIKE ON TUESDAY?
high_cardinality = train.describe(exclude='number').T.index.tolist()
high_cardinality.remove('BOROUGH')
features = train.columns.drop([target] + high_cardinality)

In [0]:
X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

In [117]:
# LETS INVESTIGATE SOME OF THESE COLUMNS FIRST

# NOTHING HERE, SO DROP 'EASE-MENT'
train['EASE-MENT'].value_counts()

# ALL THE SAME HERE, SO WE CAN DROP
train['TAX_CLASS_AT_TIME_OF_SALE'].value_counts()

1    2507
Name: TAX_CLASS_AT_TIME_OF_SALE, dtype: int64

In [0]:
def feature_cleaning(df):
  df = df.drop(['EASE-MENT', 'TAX_CLASS_AT_TIME_OF_SALE'], axis=1)
  return df

In [0]:
X_train = feature_cleaning(X_train)
X_test = feature_cleaning(X_test)

In [0]:
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [121]:
X_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT
44,1,0,0,0,0,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0
61,0,1,0,0,0,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0
78,0,0,1,0,0,4210,19,10461.0,1.0,0.0,1.0,2043.0,1925.0
108,1,0,0,0,0,5212,69,11226.0,1.0,0.0,1.0,2680.0,1899.0
111,1,0,0,0,0,7930,121,11203.0,1.0,0.0,1.0,1872.0,1940.0


## Feature Selection

In [0]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [123]:
for k in range(1, len(X_train.columns)+1):
  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  model = LinearRegression()
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)

  mae = mean_absolute_error(y_test, y_pred)

  print(f'Features: {k}\tMean Absolute Error: {mae:,.0f}')

Features: 1	Mean Absolute Error: 183,641
Features: 2	Mean Absolute Error: 184,337
Features: 3	Mean Absolute Error: 183,041
Features: 4	Mean Absolute Error: 183,598
Features: 5	Mean Absolute Error: 176,859
Features: 6	Mean Absolute Error: 176,264
Features: 7	Mean Absolute Error: 165,857
Features: 8	Mean Absolute Error: 166,978
Features: 9	Mean Absolute Error: 167,035
Features: 10	Mean Absolute Error: 167,035
Features: 11	Mean Absolute Error: 167,035
Features: 12	Mean Absolute Error: 167,070
Features: 13	Mean Absolute Error: 167,070


## Ridge Regression

In [0]:
from sklearn.linear_model import RidgeCV

In [125]:
alphas = [0.0001, 0.0005, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

# Optimize regularization
model = RidgeCV(alphas=alphas, normalize=True)
model.fit(X_train, y_train)
print(f'Optimized regularization: {model.alpha_}')

Optimized regularization: 0.0005


In [0]:
# Generate predictions
y_pred = model.predict(X_test)

In [127]:
# Test MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: ${mae:,.0f}')

Mean Absolute Error: $167,029


**We can see that this Mean Absolute Error is 6 dollars (167,035 v. 167,029) better than regressions above**